In [69]:
from IPython.core.display import display
from gremlin_python import statics
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.process.graph_traversal import __
from gremlin_python.structure.graph import Graph, Vertex, Edge, Element
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import Cardinality
from gremlin_python.process.traversal import Column
from gremlin_python.process.traversal import Direction
from gremlin_python.process.traversal import Operator
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Pop
from gremlin_python.process.traversal import Scope
from gremlin_python.process.traversal import Barrier
from gremlin_python.process.traversal import Bindings
from gremlin_python.process.traversal import WithOptions

from functools import reduce

import rdflib
import pathlib
import jsonpickle

In [3]:
# try:
#     g = traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))
# except :
#     display("Run server. Eg: docker run --name gremlin-server -p 8182:8182 tinkerpop/gremlin-server")

    
try:    
    graph = Graph()
    connection = DriverRemoteConnection('ws://localhost:8182/gremlin', 'g')
    # The connection should be closed on shut down to close open connections with connection.close()
    g = graph.traversal().withRemote(connection)
    # Reuse 'g' across the application
except :
    display("Run server. Eg: docker run --rm --name janusgraph-default janusgraph/janusgraph:latest")

In [215]:
# OUTPUT_FILE_LAM_PROPERTIES = pathlib.Path("../output/lam_project_properties_v2.ttl").resolve()
# gg = rdflib.Graph()
# gg.parse(str(OUTPUT_FILE_LAM_PROPERTIES), format="ttl")
# print("dfdf")

# for s, p, o in gg:
#     # find s in the property graph
#     #     existentURIlist = g.V().has("iri",str(s)).id().toList()
#     #     if existentURIlist:
#     #         v1 = g.V(existentURIlist[0])
#     #
#     # if o is literal add it as value to key p
#     # else create node o and add propoerty to o
#     v1 = create_node(g,s,gg)
#     v1 = create_node(g,o,gg)
#     #v1 = g.addV(s).property('iri',str(s) ).next()
#     #v2 = g.addV(o).property('iri',str(o) ).next()
#     #     g.V(Bindings.of('id',v1)).addE(p).to(v2).iterate()

2697


In [141]:
# g.V().drop().iterate()

from pprint import pprint


# s = g.V().has('iri','http://publications.europa.eu/resources/authority/celex/md_CODE').union(__.valueMap(), __.out().valueMap().tree()).toList()
# pprint(s)

# s = g.V().has('iri','http://publications.europa.eu/resources/authority/celex/md_CODE').outE().inV().tree().next()

# pprint(s)
# pprint( s["@value"][0]["value"]["@value"][0]["key"].label  )
# ss = g.V(s["@value"][0]["key"]).propertyMap().next()  
# print (ss["dct:created"][0].value)


# {'@type': 'g:Tree',
#  '@value': [{'key': v[309],
#              'value': {'@type': 'g:Tree',
#                        '@value': [{'key': v[337],
#                                    'value': {'@type': 'g:Tree', '@value': []}},
#                                   {'key': v[386],
#                                    'value': {'@type': 'g:Tree', '@value': []}},
#                                   {'key': v[349],
#                                    'value': {'@type': 'g:Tree',
#                                              '@value': []}}]}}]}


def flatten_list_of_dicts(list_of_dicts):
    """
        Flatten the list of dictionaries:
        This is an expression for Python 3.5 or greater that merges dictionaries using reduce
        Warning: repeating kets will be overwitten.
    """
    return reduce(lambda x, y: {**x, **y}, list_of_dicts, {})


def node_properties(node, g):
    """
        return the node propoerties as simple dictionary
    """
    props = g.V(node).propertyMap().next()
    # reduction of props to simple dict
    props = {key: [e.value for e in props[key]] for key in props }
    # deflating the values from lists to simple values if there is a single element provided
    return {key: props[key] if len(props[key])>1 else props[key][0] for key in props}
    
    
def expand_tree(tree_dict, g):
    """
        given a tree with a single root node, return the simple dict corresponding to the tree
        
        expecting a gremlin tree of vertices/edges, for example:
        g.V(233).outE().inV().tree().next()
    """
    
    if "@type" in tree_dict and "@value" in tree_dict:        
        if not tree_dict["@value"]: 
            # reached a leaf
            return
        else:
            # iterate the tree nodes on 'key' and their children on 'value'
#             expected_list_of_edge_dicts = [expand_tree(kv,g) for kv in tree_dict["@value"]]            
            expected_list_of_dicts = [expand_tree(kv,g) for kv in tree_dict["@value"]]            
            
            # prime if the list of dicts are edges or nodes, nodes have '@id' and '@label' property while edges don't
            if expected_list_of_dicts and "@id" in expected_list_of_dicts[0]:
                # we have a list of node descriptions
                return expected_list_of_dicts
            else:
                # we have a list od edges, which need to be flattened because in the upper step they 
                # are merged with the data properties of the node
                return flatten_list_of_dicts(expected_list_of_dicts)
                 
    elif "key" in tree_dict and "value" in tree_dict:
        # process edge
        if isinstance(tree_dict["key"], Edge):
            return {tree_dict["key"].label : expand_tree(tree_dict["value"], g)}
        # process node (expand the node propeorties and merge with edges)
        elif isinstance(tree_dict["key"], Vertex):            
#             props = {k:(v[0].value if v[0] else None) for k,v in props}
            props = node_properties(tree_dict["key"],g)
            props["@id"] = tree_dict["key"].id
            props["@label"] = tree_dict["key"].label
            
            expected_dict_of_edge_props = expand_tree(tree_dict["value"], g)            
            if expected_dict_of_edge_props:
                # deflating the values from lists to simple values if there is a single element provided            
                expected_dict_of_edge_props = {key: expected_dict_of_edge_props[key] if len(expected_dict_of_edge_props[key])>1 else expected_dict_of_edge_props[key][0] for key in expected_dict_of_edge_props}
                return {**props, **expected_dict_of_edge_props}
            else:
                return props
            
            
        


In [144]:
s = g.V().has('iri','http://publications.europa.eu/resources/authority/celex/md_CODE').outE().inV().tree().next()
# s = g.V().outE().inV().tree().next()
pprint(s)
j = expand_tree(s,g)
pprint(j)

{'@type': 'g:Tree',
 '@value': [{'key': v[309],
             'value': {'@type': 'g:Tree',
                       '@value': [{'key': e[400][309-skos:inScheme->337],
                                   'value': {'@type': 'g:Tree',
                                             '@value': [{'key': v[337],
                                                         'value': {'@type': 'g:Tree',
                                                                   '@value': []}}]}},
                                  {'key': e[388][309-sh:path->386],
                                   'value': {'@type': 'g:Tree',
                                             '@value': [{'key': v[386],
                                                         'value': {'@type': 'g:Tree',
                                                                   '@value': []}}]}},
                                  {'key': e[399][309-rdf:type->349],
                                   'value': {'@type': 'g:Tree',
                   